In [ ]:
%pip install torch numpy matplotlib opencv-python boto3 cryptography

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from src.s3_utils import download_model_weights
from src.decrypt_utils import decrypt_weights

def load_model_weights(weight_path, s3_bucket=None, s3_key=None):
    if s3_bucket and s3_key:
        try:
            encrypted_weights_path = download_model_weights(s3_bucket, s3_key)
            weight_path = decrypt_weights(encrypted_weights_path)
        except Exception as e:
            print(f"Failed to load weights from S3: {e}")
            print("Falling back to local weights...")
    return weight_path

samples_path = '/Users/raheelzubairi/Documents/projects/pixelence/v2/_samples_/'
weight_path = load_model_weights(
    '/Users/raheelzubairi/Documents/projects/pixelence/v2/_weights_/_global1.pth',
    s3_bucket='your-s3-bucket-name',
    s3_key='path/to/encrypted_weights.pth'
)  # Update with your S3 bucket and key

device = 'cuda' if torch.cuda.is_available() else 'cpu'
size = (182, 182, 22)


In [ ]:
class ContrastEnhancementBlock3D(nn.Module):
    def __init__(self, channels):
        super(ContrastEnhancementBlock3D, self).__init__()
        self.conv1 = nn.Conv3d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(channels)
        self.conv2 = nn.Conv3d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(channels)

        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.max_pool = nn.AdaptiveMaxPool3d(1)
        self.fc1 = nn.Conv3d(channels, channels // 4, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv3d(channels // 4, channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        local = nn.Sequential(
            self.conv1, self.bn1, self.relu, self.conv2, self.bn2
        )(x)

        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        attention = self.sigmoid(avg_out + max_out)

        out = x + local * attention
        return self.relu(out)

class CBAM3D(nn.Module):
    def __init__(self, in_channels, reduction=8, spatial_kernel=7):
        super(CBAM3D, self).__init__()
        self.channel_attention = ChannelAttention3D(in_channels, reduction)
        self.spatial_attention = SpatialAttention3D(spatial_kernel)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

class UNet3D_Deep_Supervision_attention_cbam(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, base_filters=32):
        super(UNet3D_Deep_Supervision_attention_cbam, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.base_filters = base_filters

        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv3d(in_c, out_c, kernel_size=3, padding=1),
                nn.BatchNorm3d(out_c),
                nn.ReLU(inplace=True),
                nn.Conv3d(out_c, out_c, kernel_size=3, padding=1),
                nn.BatchNorm3d(out_c),
                nn.ReLU(inplace=True),
            )

        self.encoder1 = conv_block(self.in_channels, self.base_filters)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.encoder2 = conv_block(self.base_filters, self.base_filters * 2)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.enc_attent2 = CBAM3D(self.base_filters * 2)
        self.encoder3 = conv_block(self.base_filters * 2, self.base_filters * 4)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.enc_attent3 = CBAM3D(self.base_filters * 4)
        self.bottleneck = conv_block(self.base_filters * 4, self.base_filters * 8)
        self.bottleneck_attent = CBAM3D(self.base_filters * 8)
        self.up3 = nn.ConvTranspose3d(self.base_filters * 8, self.base_filters * 4, kernel_size=2, stride=2)
        self.attent3 = CBAM3D(self.base_filters * 4)
        self.decoder3 = conv_block(self.base_filters * 8, self.base_filters * 4)
        self.dec3_drop = nn.Dropout3d(0.25)
        self.ds3_out = nn.Conv3d(self.base_filters * 4, self.out_channels, kernel_size=1)
        self.up2 = nn.ConvTranspose3d(self.base_filters * 4, self.base_filters * 2, kernel_size=2, stride=2)
        self.attent2 = CBAM3D(self.base_filters * 2)
        self.decoder2 = conv_block(self.base_filters * 4, self.base_filters * 2)
        self.dec2_drop = nn.Dropout3d(0.25)
        self.ds2_out = nn.Conv3d(self.base_filters * 2, self.out_channels, kernel_size=1)
        self.up1 = nn.ConvTranspose3d(self.base_filters * 2, self.base_filters, kernel_size=2, stride=2)
        self.attent1 = CBAM3D(self.base_filters)
        self.decoder1 = conv_block(self.base_filters * 2, self.base_filters)
        self.dec1_drop = nn.Dropout3d(0.25)
        self.output_conv = nn.Conv3d(self.base_filters, self.out_channels, kernel_size=1)

    def forward(self, x):
        # Forward pass implementation
        pass


In [ ]:
files = [samples_path+i for i in os.listdir(samples_path)]
for f_ in range(len(files)):
    print(f'Case {f_}')
    show_example(files[f_])
